In [13]:
from tem_dataloader import MultimodalDataset

dataset = MultimodalDataset('/media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy')

Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P000
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P001
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P002
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P003
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P004
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P005
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P006
Processing folder: /media/tyz/3B6FFE7354FF3296/11_777/tartanairv2filtered/AbandonedCableExposure/Data_easy/P007
Number of images: 25448
Number of depth: 25448
Number of lidar: 25448


In [14]:
from torch.utils.data import DataLoader, Dataset
batch_size = 32
train_dataloader = DataLoader(dataset, batch_size=batch_size, num_workers=4, shuffle=True)

In [15]:
from custom_models.CustomViT import CustomViT
from custom_models.CustomViTMAE import CustomViTMAE
import torch
# call CustomViT
from transformers import AutoImageProcessor, ViTMAEForPreTraining, ViTMAEConfig
from PIL import Image

output_dir='/home/tyz/Desktop/11_777/camelmera/weights/'
model = CustomViTMAE.from_pretrained(output_dir)
# model_name = "facebook/vit-mae-base"

# vit_config = ViTMAEConfig.from_pretrained(model_name)
# vit_config.output_hidden_states=True
# vit_model = CustomViT.from_pretrained(model_name,config=vit_config)


# model_name = "facebook/vit-mae-base"

# config = ViTMAEConfig.from_pretrained(model_name)
# config.output_hidden_states=True

# # load from pretrained model and replace the original encoder with custom encoder
# custom_model = CustomViTMAE.from_pretrained("facebook/vit-mae-base",config=config)
# custom_model.vit = vit_model

Some weights of the model checkpoint at /home/tyz/Desktop/11_777/camelmera/weights/ were not used when initializing CustomViTMAE: ['vit.embeddings1.cls_token', 'vit.embeddings1.patch_embeddings.projection.bias', 'vit.mlp.2.weight', 'vit.embeddings1.position_embeddings', 'vit.embeddings2.position_embeddings', 'vit.embeddings2.cls_token', 'vit.mlp.0.bias', 'vit.embeddings2.patch_embeddings.projection.weight', 'vit.mlp.2.bias', 'vit.embeddings1.patch_embeddings.projection.weight', 'vit.embeddings2.patch_embeddings.projection.bias', 'vit.mlp.0.weight']
- This IS expected if you are initializing CustomViTMAE from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CustomViTMAE from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassific

In [16]:
def train(model, dataloader, optimizer, scheduler, device):
    model.train()
    total_loss = 0
    total_samples = 0
    
    for batch in dataloader:
        pixel_values = batch["pixel_values"].to(device)
        pixel_values1 = batch["pixel_values1"].to(device)
        pixel_values2 = batch["pixel_values2"].to(device)
        # batch = {k: v.to(device) for k, v in batch.items()}

        optimizer.zero_grad()
        outputs = model(pixel_values,pixel_values1,pixel_values2)
        loss = outputs.loss
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

        total_loss += loss.item() * pixel_values.size(0)
        total_samples += pixel_values.size(0)

        if total_samples % 100 == 0:
          print(loss.item())

    return total_loss / total_samples


In [17]:
from transformers.optimization import AdamW
from transformers import get_linear_schedule_with_warmup

epochs = 5
learning_rate = 0.01
weight_decay = 0.01
warmup_steps = 0
total_steps = len(train_dataloader) * epochs
optimizer = AdamW(custom_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
custom_model.to(device)


cuda


/home/tyz/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


CustomViTMAE(
  (vit): CustomViT(
    (embeddings): ViTMAEEmbeddings(
      (patch_embeddings): ViTMAEPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
    )
    (encoder): ViTMAEEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTMAELayer(
          (attention): ViTMAEAttention(
            (attention): ViTMAESelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTMAESelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTMAEIntermediate(
            (dense): Linear(in_features=768, out_features=3072, bias=True)
 

In [18]:
for epoch in range(epochs):
    train_loss = train(custom_model, train_dataloader, optimizer, scheduler, device)
    print(f"Epoch: {epoch + 1}, Loss: {train_loss:.4f}")
    output_dir='/home/tyz/Desktop/11_777/camelmera/weights/'
    custom_model.save_pretrained(output_dir)

In [ ]:
output_dir='/home/tyz/Desktop/11_777/camelmera/weights/'
custom_model.save_pretrained(output_dir)